# 05-06 (or 05-13?)
Interactive julia/numerical methods tutorial

https://epubs.siam.org/doi/abs/10.1137/141000671

Need to be more focused on examples, provide less details and just have references for good reading maybe?

Implicit-explicit, spectral TDGL solver.

## Problem statement
Consider the TDGL equation:
$$\partial_t\psi=\psi+(1+ib)\partial_x^2\psi-(1+ic)|\psi|^2\psi$$
with $\psi=\psi(x,t)$ being the order parameter defined on the interval $x \in [0,L]$. Consider Dirichlet boundary conditions $\psi(0,t)=\psi(L,t) = 0$. Assume $L = 300$, $b = 0.5$, and $c=-1.76$.

Goal: write an implicit-explicit Euler method to solve this equation given an initial condition.

## Setup

Import required packages and define our parameters

In [ ]:
import Pkg; Pkg.add("BenchmarkTools")

In [ ]:
using ProgressMeter
using BenchmarkTools
using FFTW
using LinearAlgebra
using SparseArrays
using Plots
using LaTeXStrings

L = 300
b = 0.5
c = -1.76

## Discretization in time
Let $\psi^n(x) = \psi(x,t_0+n\Delta t)$, where $\Delta t$ is the size of the timestep.
### Implicit/explicit timestepping methods
Consider the general PDE:
$$\partial_t\psi = f(\psi)$$
In the following, we know the value of the unknown $\psi$ at the current timestep $\psi^n = \psi(t_0+n\Delta t)$, and want to determine the value of the unknown at the subsequent timestep $\psi^{n+1} = \psi(t_0 + (n+1)\Delta t)$. We approximate the differential operator $\partial_t$ with the following expression:
$$\partial_t\psi \approx \frac{\psi(t+\Delta t)-\psi(t)}{\Delta t}$$
#### Explicit (forward-Euler) method:
$$\psi^{n+1} - \psi^n = \Delta tf(\psi^n)$$
#### Implicit (backward-Euler) method:
$$\psi^{n+1} - \psi^n = \Delta tf(\psi^{n+1})$$
#### Implicit (trapezoidal) method:
$$\psi^{n+1} - \psi^n = \frac{1}{2}\Delta t\left[f(\psi^{n+1})+f(\psi^n)\right]$$
The allure of the explicit forward-Euler method is that the solve is trivial, since we don't have to find the roots of $f$. This is not the case for implicit methods. However, if $f$ is linear, i.e. we can write $f(\psi) = \mathcal{L}\psi$ where $\mathcal{L}$ is some linear operator (a matrix or differential/integral operator), it is still quite straightforward to implement. Let's consider the backwards-Euler method:
$$\psi^{n+1}-\psi^n = \Delta t\mathcal{L}\psi^{n+1}$$
Rearranging terms, we have a linear equation for $\psi^{n+1}$ with a right hand side of $\psi^n$:
$$(1-\Delta t\mathcal{L})\psi^{n+1} = \psi^n$$
This can be solved with a direct solver (e.g. LU factorization) and can be quite fast (i.e. linear in the number of elements of $\psi$) depending on the properties of $\mathcal{L}$. The update equation for the trapezoidal method looks very similar:
$$(2 - \Delta t \mathcal{L})\psi^{n+1} = (2 + \Delta t \mathcal{L})\psi^{n+1}$$
If $f$ is nonlinear, however, things get a bit more difficult. One can continue to do a full implicit solve, where one must resort to a nonlinear root-finding method, such as Newton. However, it's also possible to split $f$ into $f(\psi) = \mathcal{L}\psi + f_{N\mathcal{L}}(\psi)$, where $\mathcal{L}$ is a linear operator and $f_{N\mathcal{L}}$ contains the nonlinear part. The implicit-explicit method assumes that the timestep is sufficiently small that $f(\psi^{n+1}) \approx \mathcal{L}\psi^{n+1}+f_{N\mathcal{L}}(\psi^n)$.

Therefore, the implicit-explicit update equation is:
$$(1 - \Delta t\mathcal{L})\psi^{n+1} = \psi^n + \Delta tf_{N\mathcal{L}}(\psi^n)$$
This has the advantage of not requiring a Netwon method, while still retaining some of the nice qualities of an implicit method, like allowing for larger timesteps.

## Discretization in space
Now that we have a linear equation to solve to update our unknown for each timestep $\psi^n(x)$, we need to discretize the spatial operator $\mathcal{L}$.

### Chebyshev polynomials, fast transforms, and the Petrov-Galerkin method
The Chebyshev polynomials are related to the Fourier basis functions in that $T_n = cos(n\Theta)$ where the trigonometric coordinate $\Theta$ is related to the spatial coordinate $x$ by $x = \cos\Theta$.
We can plot the first few Chebyshev polynomials

In [ ]:
plot()
for n=0:3
    plot!(range(-1,1,1000), cos.(n.*acos.(range(-1,1,1000))), label=L"T_%$n(x)")
end
plot!()

We will expand the solution $\psi^{n+1}(x)$ in terms of a trial basis of Chebyshev polynomials:
$$\psi^n(x)\approx\sum_{k=0}^{N-1}\hat{\psi^n_k}T_k(x)$$
We can define our unknown vector in terms of these coefficients $\psi^n=\begin{pmatrix}\hat\psi^n_0&\ldots&\hat\psi^n_{N-1}\end{pmatrix}^T$.

Because of the relationship between Chebyshev polynomials and the Fourier basis (Chebyshev polynomials are just Fourier basis functions in disguise), we can leverage Fast Fourier Transforms to go between $\hat\psi^n_k$ and $\psi^n(x_k)$:

In [ ]:
function chebyshev_nodes(N_grid::Int64)::Vector{Float64}
    x_n = cos.(pi / (N_grid - 1) .* (0:(N_grid-1)))
    reverse!(x_n)
    return x_n
end

function chebyshev_coeffs!(u_n::Vector{<:Number}, pDCT1::FFTW.Plan)
    # pDCT1 = FFTW.r2r(u_n, FFTW.REDFT00, 1)
    # reverse isn't strictly necessary, but it's nicer for plotting
    reverse!(u_n)
    pDCT1 * u_n # in place
    u_n ./= length(u_n) - 1.0
    # c_0, c_(N-1) = 1/2, c_k = 1 otherwise
    u_n[1] /= 2.0
    u_n[end] /= 2.0
    nothing
end

function chebyshev_values!(uhat_k::Vector{<:Number}, pDCT1::FFTW.Plan)
    uhat_k[1] *= 2.0
    uhat_k[end] *= 2.0
    # pDCT1 = FFTW.r2r(uhat_k, FFTW.REDFT00, 1)
    pDCT1 * uhat_k
    uhat_k ./= 2.0
    # reverse isn't strictly necessary, but it's nicer for plotting
    reverse!(uhat_k)
    nothing
end

Let's use these functions to plot the Chebyshev polynomials again:

In [ ]:
N = 128
x_n = chebyshev_nodes(N)
d = zeros(N)
p_g2c = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p_c2g = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p = plot()
for m = 0:3
    d = zeros(N)
    d[m+1] = 1
    chebyshev_values!(d, p_c2g)
    plot!(p, x_n, d, label=L"T_%$m(x)")
end
xlabel!(p, L"x")
ylabel!(p, L"T_n(x)")
plot(p)

If we plot the error compared to the analytical expression for $T_n(x)$, we can see we're at machine precision:

In [ ]:
p = plot()
for m = 0:3
    d = zeros(N)
    d[m+1] = 1
    chebyshev_values!(d, p_c2g)
    plot!(p, x_n, d.-cos.(m.*acos.(x_n)))
end
xlabel!(p, L"x")
ylabel!(p, "error")
plot(p, legend=false)

After expanding our unknown in terms of Chebyshev polynomials, we are left with a finite number of equations to solve, however they must be translated into a weak form, since we cannot use a computer to enforce our PDE to hold for every value of $x$.
In the weak form, we take an inner product over every element of a finite-dimensional test basis and only require that our PDE be satisfied over these inner products. Considering a PDE of the form $\mathcal{L}u = f$ with unknown $u$, forcing term $f$, and differential operator $\mathcal{L}$, this translates into the following set of equations:
$$\begin{aligned}\langle w,\mathcal{L}u\rangle = \langle w,f\rangle \\\forall w \in W_N\end{aligned}$$
where $w$ is any test basis belonging to the finite-dimensional vector space $W_N$.

By carefully selecting ultraspherical/Gegenbauer polynomials for our test basis $W_N$ and Chebyshev polynomials for our trial basis, we can ensure that the resulting matrix equation will be sparse.

### Constructing differentiation and conversion operators
Using the inner product: $\langle\partial_xT_n(x),C^{(1)}_{m}(x)\rangle=n\delta_{m,n-1}$ and relationship between successive derivatives of ultraspherical polynomials, we get the following differentiation matrix when operating on our unknown vector of Chebyshev coefficients:
$$D_{\lambda}=2^{\lambda-1}(\lambda-1)!\begin{bmatrix}\overbrace{0 ... 0}^{\lambda~{\rm times}} & 1 &\\&& 2 \\&&& 3\\&&&&\ddots\end{bmatrix}, \lambda>0$$
We can implement this in Julia as so:

In [ ]:
function D(N::Int64, lambda::Int64)::SparseMatrixCSC{Float64,Int}
    return (2^(lambda - 1) * factorial(lambda - 1)) .* spdiagm(lambda => lambda:(N-1))
end

Conversion between Chebyshev coefficients and the various ultraspherical coefficients can be done through the following matrices
$$\begin{aligned}S_{\lambda}&=\begin{bmatrix}1 & &-\frac{\lambda}{\lambda+2}\\&\frac{\lambda}{\lambda+1}&&-\frac{\lambda}{\lambda+3}\\&&\frac{\lambda}{\lambda+2}&&-\frac{\lambda}{\lambda+4}\\&&&\ddots&&\ddots\end{bmatrix},\lambda>1\\S_0&=\begin{bmatrix}1 & & -\frac{1}{2}\\&\frac{1}{2}&&-\frac{1}{2}\\&&\frac{1}{2}&&-\frac{1}{2}\\&&&\ddots&&\ddots\end{bmatrix}\end{aligned}$$
Which has the following Julia implementation

In [ ]:
function S(N::Int64, lambda::Int64)::SparseMatrixCSC{Float64,Int}
    if lambda >= 1
        return spdiagm(
            0 => lambda ./ (lambda .+ collect(0:(N-1))),
            2 => -lambda ./ (lambda .+ collect(2:(N-1))),
        )
    end
    S0 = spdiagm(0 => ones(N), 2 => -ones(N - 2))
    S0[1, 1] = 2
    return S0 ./ 2
end

We can now use some of the functions we've created to take some derivatives of basic functions. Take for example the function
$$f(x)=\frac{1}{1+10x^2}$$

In [ ]:
N = 64
x_n = chebyshev_nodes(N)
d = zeros(N)
p_g2c = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p_c2g = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p = plot()
f = tanh.(x_n*2)
plot!(p,x_n, f, label=L"\tanh(x)")
chebyshev_coeffs!(f, p_g2c)
dfdx = S(N, 0) \ (D(N, 1) * f)
chebyshev_values!(dfdx, p_c2g)
plot!(p,x_n,dfdx, label=L"\frac{d}{dx}\tanh(x)")
xlabel!(p, L"x")
ylabel!(p, L"f")
plot(p)

In [ ]:
plot(x_n,dfdx-2*sech.(2*x_n).^2,label="error")
xlabel!(L"x")

Let's compare with a first-order finite-difference method

In [ ]:
x_n = range(-1,1,N)
f = tanh.(x_n*2)
dfdx = (f[3:end] - f[1:end-2])./(2*(x_n[2] - x_n[1]))
plot(x_n, 2*sech.(2*x_n).^2, label="analytical")
plot!(x_n[2:end-1], dfdx, label="finite difference")
xlabel!(L"x")

In [ ]:
plot(x_n[2:end-1], dfdx - 2*sech.(2*x_n[2:end-1]).^2,label="error")
xlabel!(L"x")

The error for the finite-difference method is nearly on the order of 1%, whereas the error for the spectral method is approaching machine precision.
How different are the runtimes?

In [ ]:
function dfdx_fd(f::Vector{Float64}, dx::Float64)::Vector{Float64}
    return (f[3:end] - f[1:end-2])./(2*dx)
end

function dfdx_us(f::Vector{Float64}, p_g2c::FFTW.Plan, p_c2g::FFTW.Plan, S0::SparseMatrixCSC{Float64,Int}, D1::SparseMatrixCSC{Float64,Int})::Vector{Float64}   
    chebyshev_coeffs!(f, p_g2c)
    dfdx = S0 \ (D1 * f)
    chebyshev_values!(dfdx, p_c2g)
    return dfdx
end

N = 256
x_n = chebyshev_nodes(N)
d = zeros(N)
p_g2c = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p_c2g = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
dx = x_n[2] - x_n[1]
S0 = S(N, 0)
D1 = D(N, 1)

f = tanh.(x_n*2)
@btime dfdx_fd($f, $dx)
f = tanh.(x_n*2)
@btime dfdx_us($f, $p_g2c, $p_c2g, $S0, $D1)



## Relationship with generalized TDGL
If we consider for simplicity $\gamma = \mu = 0$ and $\mathbf{A} = 0$, as well as $\epsilon = 1$, then the generalized TDGL equation becomes
$$\partial_t\psi=\psi-|\psi|^2\psi+\partial_x^2\psi$$
which is identical to what we'd written previously with $b = c = 0$.